# Aplicar preprocesado y formatear texto
En este notebook vamos a aplicar el preprocesado que hemos estudiado en el notebook anterior al corpus que tenemos. Tras ello, vamos a pasar el texto a formato conllu, que es lo que el modelo que usamos para resolver las referencias usa como entrada. Esto lo hemos dividido así porque nos ahorramos guardar el dataset etiquetado, ya lo guardamos todo en formato conllu.  
Para el preprocesado que le vamos a aplicar, vamos a guardarlo en un archivo .py para mejorar la legibilidad y paralelizar el código para que lo haga un poco más rápido. Además, esas funciones ya las definimos en el notebook de preprocesar, por lo que simplemente copiamos aquellas funciones que usamos.

# 1 Aplicar preprocesado

## 1.1 Imports
Vamos a poner todos los imports que necesitamos para aplicar el preprocesado.

In [1]:
import json
import preprocesar
import stanza

In [ ]:
stanza.download('es')

## 1.2 Funciones auxiliares
Ahora vamos a definir nuestra función para preprocesar el texto y todas sus sub-funciones. Recordemos que primero vamos a pasarle nuestro tokenizador personalizado y despues el modelo stanza para sacar las part-of-speech-tags.

In [2]:
def print_tags(docs: list = []):
    for sentence in docs.sentences:
        for token in sentence.words:
            print(token.text, token.pos)
        print()    

In [3]:
with open('CulturaX.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
  
# Cogemos un subset
dt = data[:500]
# Le pasamos solo los textos
texts = [d['text'] for d in dt]

In [4]:
doc, imperative_meta = preprocesar.preprocesar_paralelo(texts)

Preprocesando:   0%|          | 0/500 [00:00<?, ?it/s]

In [63]:
# Mostrar las etiquetas
for d in doc[:2]:   
    print_tags(d)
    print('-'*20)

gobierno NOUN
confía VERB
en ADP
que SCONJ
corte NOUN
constitucional ADJ
dé VERB
vía NOUN
libre ADJ
a ADP
el DET
mecanismo NOUN
fast PROPN
track PROPN
- PUNCT
eje21 ADJ
gobierno NOUN
confía VERB
en ADP
que SCONJ
corte NOUN
constitucional ADJ
dé VERB
vía NOUN
libre ADJ
a ADP
el DET
mecanismo NOUN
fast PROPN
track PROPN
bogotá PROPN
, PUNCT
04 NUM
de ADP
diciembre PROPN
_ PUNCT
ram_ NOUN
así ADV
lo PRON
confirmó VERB
este DET
sábado NOUN
el DET
ministro NOUN
de ADP
el DET
interior NOUN
juan PROPN
fernando PROPN
cristo PROPN
, PUNCT
quien PRON
señaló VERB
que SCONJ
: PUNCT
" PUNCT
el DET
gobierno NOUN
emprenderá VERB
el DET
camino NOUN
de ADP
implementación NOUN
de ADP
el DET
nuevo ADJ
acuerdo NOUN
de ADP
paz NOUN
que PRON
decida VERB
la DET
corte NOUN
constitucional ADJ
" PUNCT
, PUNCT
y CCONJ
confía VERB
en ADP
que SCONJ
la DET
decisión NOUN
sobre ADP
la DET
vía NOUN
rápida ADJ
para ADP
la DET
implementación NOUN
de ADP
las DET
leyes NOUN
de ADP
paz NOUN
se PRON
tome VERB
antes ADV
de A

# 2 Pasar a formato conllu
Ahora vamos a pasar los datos que hemos sacado antes a formato conllu. Para ello vamos a definir unas funciones auxiliares que nos ayudarán. Vamos a mencionar cómo se construye el formato conllu:
1. Se incluyen líneas comentadas con el documento al que pertenece la frase, un identificador que nos diga que frase es dentro del documento y la frase original antes del tokenizado. <br><br>
2. Una linea para cada palabra que constará de 10 columnas:
 - **ID**: Índice del token. Su posición dentro de la oración.
 - **FORM**: Forma de la palabra. Palabra que aparece tal cual en el texto original.
 - **LEMMA**: Forma base de la palabra.
 - **UPOS**: Categoría gramatical universal de la palabra.
 - **XPOS**: Etiqueta gramatical específica del idioma.
 - **FEATS**: Lista de rasgos morfológicos (género, número, tiempo, persona).
 - **HEAD**: El ID de la palabra de la que depende.
 - **DEPREL**: El tipo de relación con el HEAD.
 - **DEPS**: Grafo de dependencias mejorado.
 - **MISC**: Cualquier otra información.

## 2.1 Imports

In [13]:
from tqdm.notebook import tqdm

## 2.2 Funciones auxiliares

**stanza_doc_to_conllu**

Entrada: 
- Texto tokenizado por stanza
- Lista de palabras a las que hay que cambiarles algún campo

Salida: Texto en formato conllu

Recorreras cada oración analizada por stanza y escribirás los encabezados que identifican cada frase. Después se cambiarán de forma manual los lemmas y upos de las palabras que coincidieron con algún patrón de la lista de excepciones para que no se pierda el significado.

Una vez hecho esto, el código recolectará los atributos del análisis que stanza hizo previamente y los dispondrá en el formato conllu final

In [5]:
def stanza_doc_to_conllu(doc, imperative_meta) -> str:
    lines = []
    sent_id = 1

    for sent in doc.sentences:
        # Identificadores de cada frase
        lines.append(f"# sent_id = {sent_id}")
        lines.append(f"# text = {sent.text}")

        for word in sent.words:
            lemma = word.lemma
            upos = word.upos

            # Corrección manual del lema si viene de whitelist
            for meta in imperative_meta:
                if word.text.lower() == meta["base"]:
                    lemma = meta["lemma"]
                    upos = meta["upos"]

            # Extracción de los atributos de las palabras
            feats = word.feats if word.feats else "_"

            misc = []
            if word.start_char is not None and word.end_char is not None:
                misc.append(f"CharOffset={word.start_char}:{word.end_char}")
            misc = "|".join(misc) if misc else "_"

            # Construcción del conllu final
            lines.append("\t".join([
                str(word.id),        # ID
                word.text,           # FORM
                lemma or "_",         # LEMMA (corregido)
                upos or "_",          # UPOS
                word.xpos or "_",     # XPOS
                feats,                # FEATS
                str(word.head),       # HEAD
                word.deprel or "_",   # DEPREL
                "_",                  # DEPS
                misc                  # MISC
            ]))

        lines.append("")
        sent_id += 1

    return "\n".join(lines)

FUNCIÓN PRUNCIPAL: TEXTO(S) → CoNLL-U

**texts_to_conllu**

Esta celda es la función principal que irá llamando a las funciones de las celdas anteriores.

 1º- Sustituye los verbos con clíticos por palabras separadas. Ej: dímelo -> di + me + lo

 2º- Tokenizarás cada palabra con stanza.pipeline

 3º- Construye la salida del preprocesado en un texto formato conllu

In [14]:
def texts_to_conllu(docs, imperative_metas):
    conllu_list = []; i = 1
    
    # Usar tqdm en el zip con descripción personalizada
    for doc, imperative_meta in tqdm(zip(docs, imperative_metas), 
                                                    total=len(docs), 
                                                    desc="Procesando documentos"):
        # Dar formato conllu a los tokens de la frase
        conllu = stanza_doc_to_conllu(doc, imperative_meta)
        
        # Separación entre frases
        conllu_list.append(f"# newdoc id = doc_{i}")
        conllu_list.append(conllu)
        i+=1
    
    return "\n".join(conllu_list)

## 2.3 Sacar conllu

In [15]:
conllu_output = texts_to_conllu(doc, imperative_meta)

Procesando documentos:   0%|          | 0/500 [00:00<?, ?it/s]

Vamos a mostrar un poco la salida

In [17]:
print(len(conllu_output))
print(conllu_output[:5000])

26427677
# newdoc id = doc_1
# sent_id = 1
# text = gobierno confía en que corte constitucional dé vía libre al mecanismo fast track - eje21
gobierno confía en que corte constitucional dé vía libre al mecanismo fast track
bogotá, 04 de diciembre _ ram_ así lo confirmó este sábado el ministro del interior juan fernando cristo, quien señaló que: "el gobierno emprenderá el camino de implementación del nuevo acuerdo de paz que decida la corte constitucional", y confía en que la decisión sobre la vía rápida para la implementación de las leyes de paz se tome antes de la vacancia judicial y el fin de esta legislatura en el congreso, "tenemos la fe de que va a dar salidas que garanticen implementación rápida de los acuerdos".
1	gobierno	gobierno	NOUN	ncms000	Gender=Masc|Number=Sing	2	nsubj	_	CharOffset=0:8
2	confía	confiar	VERB	vmip3s0	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin	0	root	_	CharOffset=9:15
3	en	en	ADP	sps00	_	7	mark	_	CharOffset=16:18
4	que	que	SCONJ	cs	_	7	mark	_	CharO

## 2.4 Guardar la salida
Guardamos ahora la salida del text_to_conllu en un archivo (entrada.conllu) que se usará como entrada del modelo a entrenar.

In [18]:
file_name = "entrada.conllu"

with open(file_name, "w", encoding="utf-8") as f:
    f.write(conllu_output)

print(f"Archivo guardado como {file_name}")

Archivo guardado como entrada.conllu
